In [1]:
# Каким образом Ваня собрал ноутбуки?
# Что будет, если джойнить чанки с соревнованиями, которые Ваня собрал в конце?

In [2]:
import gc

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
connection_string_path = '../../.vscode/mariadb_connection.txt'
connection_string = open(connection_string_path, 'r').read()
engine = create_engine(connection_string)

In [4]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()
for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print(table_name)
        # for column in inspector.get_columns(table_name, schema=schema):
        #     print("Column: %s" % column)

schema: #mysql50#lost+found
schema: information_schema
schema: mysql
column_stats
columns_priv
db
event
func
general_log
global_priv
gtid_slave_pos
help_category
help_keyword
help_relation
help_topic
index_stats
innodb_index_stats
innodb_table_stats
plugin
proc
procs_priv
proxies_priv
roles_mapping
servers
slow_log
table_stats
tables_priv
time_zone
time_zone_leap_second
time_zone_name
time_zone_transition
time_zone_transition_type
transaction_registry
schema: nl2ml
all_chunks
all_notebooks
almost_good_codeblocks
chunks
chunks_data
codeblocks
competitions
data_prep4notebooks
good_competitions
good_notebooks
good_notebooks_data
graph_vertexes_m
graph_vertices
history
howmany_notebooks
id2comp_id
migration_chunks
migration_n
newchunks
not_good_enough_codeblocks
notebooks
notebooks_with_ref
old_competitions
raw_chunks_data
raw_data
ref_links
results
users
schema: nl2ml_test
chunks_data
chunks_data_m
codeblocks
competition_links
competitions
notebooks
schema: performance_schema
accounts
con

In [101]:
query = """
select * from chunks_data
"""
data = pd.read_sql_query(query, engine)
data.shape[0], data.kaggle_id.nunique(), data.ref_link.nunique()

(621614, 23426, 553)

In [102]:
query = """
select * from competitions
"""
competitions_db = pd.read_sql_query(query, engine)
competitions_db.shape[0]

983

In [103]:
competitions_db.head()

,id,ref_link,comp_name,comp_type,description,metric,datatype,subject,problemtype,good,old_id
0,1,c/roxie2,'`Predict the missing pixel value v2`',inclass,'`A small regression problem.`',mse,None,None,None,0,3175.0
1,2,c/recruit-restaurant-visitor-forecasting,'`Recruit Restaurant Visitor Forecasting`',featured,'`Running a thriving local restaurant isn't al...,rmsle,None,None,None,1,NaN
2,3,c/rsna-pneumonia-detection-challenge,'`RSNA Pneumonia Detection Challenge`',featured,"'`In this competition, youre challenged to bui...",custom metric,None,None,None,1,NaN
3,4,c/reducing-commercial-aviation-fatalities,'`Reducing Commercial Aviation Fatalities`',playground,'`Most flight-related fatalities stem from a l...,multiclassloss,None,None,None,1,NaN
4,5,c/rossmann-store-sales,'`Rossmann Store Sales`',featured,"'`Rossmann operates over 3,000 drug stores in ...",rootmeansquarepercentageerror,None,None,None,1,352.0


In [99]:
query = """
select * from notebooks
"""
notebooks = pd.read_sql_query(query, engine)
notebooks.shape[0]

23394

In [112]:
notebooks = notebooks.merge(competitions_db, left_on='competition_id', right_on='id', how='inner').drop_duplicates()

In [115]:
notebooks['ref_link'].nunique()

218

In [2]:
pd.set_option('mode.chained_assignment', None)

In [78]:
MARKUP_DATA_PATH = "../../data/markup_data_2021-05-26.csv"
markup_data = pd.read_csv(MARKUP_DATA_PATH)#, nrows=25)
print('Loaded: {} chunks of {} notebooks from {} competitions'.format(markup_data.shape[0], markup_data['kaggle_id'].nunique(), markup_data['competition_id'].nunique()))

In [ ]:
NOT_YET_MARKUP_DATA_PATH = "../../data/not_yet_markup_data_2021-05-26.csv"
not_yet_markup_data = pd.read_csv(NOT_YET_MARKUP_DATA_PATH)#, nrows=25)
# not_yet_markup_data.head()
print('Loaded: {} chunks of {} notebooks from {} competitions'.format(not_yet_markup_data.shape[0], not_yet_markup_data['kaggle_id'].nunique(), not_yet_markup_data['competition_id'].nunique()))
# new_notebooks.rename({'data_sources':'ref'}, axis=1, inplace=True)
# def clean_comp(string:str) -> str:
#     string = string.strip('[').strip(']').replace("'", "")
#     return string
# new_notebooks['ref'] = new_notebooks['ref'].apply(clean_comp)
# new_notebooks['ref'] = new_notebooks['ref'].apply(lambda x: x.split(',')[0].strip(' ').strip("'"))

In [5]:
# Сколько у нас всего чанков по результатам последнего парсинга?
NEW_NOTEBOOKS_PATH = "../../data/codeblocks_2021-04-01_concatenated.csv"
new_notebooks = pd.read_csv(NEW_NOTEBOOKS_PATH)#, nrows=25)
new_notebooks.rename({'data_sources':'ref'}, axis=1, inplace=True)
print('Parsed: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))
def clean_comp(string:str) -> str:
    string = string.strip('[').strip(']').replace("'", "")
    return string
new_notebooks['ref'] = new_notebooks['ref'].apply(clean_comp)
new_notebooks['ref'] = new_notebooks['ref'].apply(lambda x: x.split(',')[0].strip(' ').strip("'"))

Parsed: 2587074 chunks of 101071 notebooks from 22935 competitions


In [6]:
# Сколько останется чанков после удаления дубликатов и чанков без кода?
new_notebooks.drop_duplicates(inplace=True, subset=['code_block_id', 'kaggle_id'])
print('After removing duplicates: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))
new_notebooks.dropna(axis=0, subset=['code_block'], inplace=True)
new_notebooks = new_notebooks[~(new_notebooks['code_block'] == '`')]
print('After dropping NaNs: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))
gc.collect()
print('Parsed and cleaned: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))

After removing duplicates: 2441865 chunks of 101065 notebooks from 12984 competitions
After dropping NaNs: 2441862 chunks of 101065 notebooks from 12984 competitions
Parsed and cleaned: 2441862 chunks of 101065 notebooks from 12984 competitions


In [7]:
new_notebooks.head()

,Unnamed: 0,kaggle_score,kaggle_comments,kaggle_upvotes,kaggle_link,kaggle_id,ref,code_block,code_block_id
0,0,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,%reload_ext autoreload\n%autoreload 2\n%matplo...,0
1,1,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,import numpy as np\nimport pandas as pd\n\nfro...,1
2,2,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,dim = 256\ntrain_files = []\ntest_files = []\n...,2
3,3,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,with open(country_file) as json_file:\n tmp...,3
4,4,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,train_set = []\nfor f in train_files:\n idx =...,4


In [9]:
# Дропаем позорище
new_notebooks.drop(['Unnamed: 0'], axis=1, inplace=True)
new_notebooks.head()

,kaggle_score,kaggle_comments,kaggle_upvotes,kaggle_link,kaggle_id,ref,code_block,code_block_id
0,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,%reload_ext autoreload\n%autoreload 2\n%matplo...,0
1,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,import numpy as np\nimport pandas as pd\n\nfro...,1
2,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,dim = 256\ntrain_files = []\ntest_files = []\n...,2
3,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,with open(country_file) as json_file:\n tmp...,3
4,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,train_set = []\nfor f in train_files:\n idx =...,4


In [13]:
# Добавляем новые будущие столбцы
new_notebooks['assessor_label'] = pd.NaT
new_notebooks['assessor_confidence'] = pd.NaT
new_notebooks['classifier_label'] = pd.NaT
new_notebooks['classifier_confidence'] = pd.NaT
new_notebooks.head()

,kaggle_score,kaggle_comments,kaggle_upvotes,kaggle_link,kaggle_id,ref,code_block,code_block_id,assessor_label,assessor_confidence,classifier_label,classifier_confidence
0,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,%reload_ext autoreload\n%autoreload 2\n%matplo...,0,NaT,NaT,NaT,NaT
1,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,import numpy as np\nimport pandas as pd\n\nfro...,1,NaT,NaT,NaT,NaT
2,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,dim = 256\ntrain_files = []\ntest_files = []\n...,2,NaT,NaT,NaT,NaT
3,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,with open(country_file) as json_file:\n tmp...,3,NaT,NaT,NaT,NaT
4,0.77609,0,1,/claudiohfg/gallivanters-public-score-0-77609,5849655,c/gallivanters,train_set = []\nfor f in train_files:\n idx =...,4,NaT,NaT,NaT,NaT


In [ ]:
# Сохраняем базу корпуса
new_notebooks.to_csv('../../corpus/MLCC_base.csv', index=False)

In [19]:
new_notebooks.dropna(axis=0, subset=['kaggle_score'], inplace=True) 
print('With kaggle_score: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))

With kaggle_score: 617441 chunks of 23425 notebooks from 553 competitions


In [20]:
new_notebooks['code_block'] = new_notebooks['code_block'].str.replace('`', '')

In [21]:
# new_notebooks.to_csv("../../data/codeblocks_2021-04-01_concatenated_cleaned.csv", index=False)

In [22]:
COMPETITIONS_PATH = "../../data/competitions_info.csv"
competitions = pd.read_csv(COMPETITIONS_PATH)
print(competitions.shape)
competitions.drop_duplicates(inplace=True)
competitions['ref'] = competitions['ref'].apply(clean_comp)
print(competitions.shape)

(5060, 8)
(4352, 8)


In [144]:
# Сколько всего у нас есть чанков с непустым kaggle_score?
CHUNKS_PATH = "../../data/codeblocks_2021-04-01_concatenated_cleaned.csv"
chunks = pd.read_csv(CHUNKS_PATH)
print('Loaded: {} chunks of {} notebooks from {} competitions'.format(chunks.shape[0], chunks['kaggle_id'].nunique(), chunks['ref'].nunique()))
print(chunks['kaggle_score'].isna().sum())
# How many competitions we have in competitions file
print('We have {} competitions in competitions file'\
    .format(chunks[chunks['ref'].isin(competitions['ref'].unique())]['ref'].nunique()))
# How many competions we do not have in competitions file
print('We miss {} competitions in competitions file'\
    .format(chunks['ref'].nunique() - chunks[chunks['ref'].isin(competitions['ref'])]['ref'].nunique()))

Loaded: 617441 chunks of 23424 notebooks from 552 competitions
0


In [23]:
new_notebooks = new_notebooks.merge(competitions, on=['ref'], how='inner')
# new_notebooks.merge(competitions, on=['ref'], how='right').shape
print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks.shape[0], new_notebooks.kaggle_link.nunique(), new_notebooks.ref.nunique()))

# Chunks: 77649
# Notebooks: 2827
# Competitions: 266


In [24]:
# Check number of "non competition" refs
new_notebooks[~new_notebooks['ref'].str.contains('c/')].shape[0]

0

In [ ]:
# new_notebooks.drop('Unnamed: 0', axis=1, inplace=True)
# new_notebooks.to_csv('../../data/codeblocks_2021-04-01_concatenated_cleaned.csv', index=False)

In [10]:
# new_notebooks['exists_in_competitions'] = new_notebooks.apply(lambda x: x['ref'] in competitions['ref'].unique(), axis=1)
# print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks[new_notebooks['exists_in_competitions'] == True].shape[0], new_notebooks[new_notebooks['exists_in_competitions'] == True]['kaggle_link'].nunique(), new_notebooks[new_notebooks['exists_in_competitions'] == True]['ref'].nunique()))

# Chunks: 77649
# Notebooks: 2827
# Competitions: 266


In [ ]:
competitions['has_notebooks'] = competitions.apply(lambda x: x['ref'] in new_notebooks['ref'].tolist(), axis=1)
competitions_cleaned = competitions[competitions['has_notebooks']]
competitions_cleaned.shape
# competitions_cleaned.to_csv('../data/competitions_info_cleaned.csv', index=False)

In [ ]:
COMPETITIONS_FILLED_PATH = "../../data/competitions_info_cleaned.csv"
competitions_filled = pd.read_csv(COMPETITIONS_FILLED_PATH)
competitions_filled.shape

In [ ]:
competitions_cleaned_with_filled = competitions_cleaned[['ref', 'comp_name', 'comp_type']].merge(competitions_filled[['ref', 'Description',
                                        'Metric', 'DataType', 'Subject', 'ProblemType']]
                                    , on=['ref'], how='left')
competitions_cleaned_with_filled.isna().sum()

In [ ]:
# competitions_filled.to_csv('../../data/competitions_info_filled.csv', index=False)
# competitions_filled.to_csv('../../data_mini/competitions_info_filled.csv', index=False)

In [ ]:
# competitions_cleaned.to_csv('../../data/competitions_info_cleaned.csv', index=False)
# competitions_cleaned.to_csv('../../data_mini/competitions_info_cleaned.csv', index=False)

In [ ]:
# competitions_cleaned_with_filled.to_csv('../../data/competitions_info_cleaned_filled.csv', index=False)
# competitions_cleaned_with_filled.to_csv('../../data_mini/competitions_info_cleaned_filled.csv', index=False)